In [1]:
# run_experiments.py
import argparse, os, json, time
from pathlib import Path
import numpy as np
import pandas as pd
from tqdm import tqdm
from datetime import datetime

from fairbench.utils.seed import set_seed
from fairbench.datasets import load_dataset   # 통합 로더
from fairbench.methods import run_method      # 통합 메서드 실행
from fairbench.metrics import compute_metrics # accuracy, supIPM, subgroup metrics


In [2]:

def parse_args():
    p = argparse.ArgumentParser(description="Subgroup Fairness Bench")

    # Dataset & data options
    p.add_argument("--dataset", type=str, required=True,
                   choices=["toy_manyq", "adult", "communities", "dutch", "celebA"])
    p.add_argument("--data_dir", type=str, default="data")
    p.add_argument("--q", type=int, default=100, help="toy_manyq sensitive count")

    # X–S handling (공통)
    p.add_argument("--x_sensitive", type=str, default="drop",
                   choices=["drop", "keep", "concat"],
                   help="drop: 민감 원본컬럼을 X에서 제거, keep: 유지, concat: S를 X에 붙여 f(x,s)로 학습")
    p.add_argument("--sens_keys", type=str, default=None,
                   help=("민감 컬럼 리스트 또는 프리셋 키워드. "
                         "Adult는 None이면 내부 기본 세트 사용, "
                         "Communities/Dutch는 None이면 각 로더의 'auto' 프리셋 사용. "
                         "예: 'race_basic' 또는 'sex,age,education_num'"))
    p.add_argument("--sens_thresh", type=float, default=0.5,
                   help="민감 수치 컬럼 이진화 분위수 임계(0~1)")

    # Dataset-specific optional paths
    p.add_argument("--dutch_path", type=str, default="data/raw/dutch.csv",
                   help="Dutch CSV 경로(기본 data/raw/dutch.csv)")
    p.add_argument("--communities_names", type=str, default=None,
                   help="communities.names 경로(기본 data_dir/raw/communities.names)")
    p.add_argument("--communities_data", type=str, default=None,
                   help="communities.data 경로(기본 data_dir/raw/communities.data)")
    p.add_argument("--tfds_data_dir", type=str, default=None,
                help="TFDS data_dir (없으면 기본 캐시)")
    p.add_argument("--celebA_manual_dir", type=str, default=None,
                help="CelebA 수동 다운로드 디렉토리(파일들 존재해야 함)")

    # Method selection
    p.add_argument("--method", type=str, required=True,
                   choices=["dr", "gerryfair", "multicalib", "sequential", "reduction"])

    # Reduction (fairlearn ExponentiatedGradient) baseline
    p.add_argument("--red_constraint", type=str, default="DP",
                   choices=["DP", "EO"], help="DP=DemographicParity, EO=EqualizedOdds")
    p.add_argument("--red_eps", type=float, default=0.02,
                   help="fairness slack (작을수록 제약 강함)")
    p.add_argument("--red_max_iter", type=int, default=50)
    p.add_argument("--red_base", type=str, default="logreg",
                   choices=["logreg", "linear_svm", "rf", "mlp_clf", "mlp_reg"],)

    # Common training hparams (필요시 각 메서드에서 선택 사용)
    p.add_argument("--seed", type=int, default=42)
    p.add_argument("--epochs", type=int, default=300)
    p.add_argument("--batch_size", type=int, default=512)
    p.add_argument("--lr", type=float, default=1e-3)

    # DR (our method)
    p.add_argument("--lambda_fair", type=float, default=0.0,
                   help="DR fairness weight (정확도-공정성 trade-off)")
    p.add_argument("--n_low", type=int, default=100,
                help="partial subgroup fairness: 최소 서브그룹 크기 임계값")
    # Data options (여기에 추가)
    p.add_argument("--shrink_smallest_frac", type=float, default=1.0,
                help="전체 n 대비 가장 작은 교차 서브그룹을 이 비율*n까지 다운샘플")
    p.add_argument("--shrink_seed", type=int, default=None,
                help="다운샘플 시드(미지정시 --seed 사용)")
    p.add_argument("--n_low_frac", type=float, default=None,
                help="partial subgroup: 최소지지 비율 (0~1). 지정 시 n_low보다 우선")


    # GerryFair
    p.add_argument("--gamma", type=float, default=0.01,
                   help="GerryFair gamma (페어니스 강도/허용 위반 수준)")
    p.add_argument("--gf_base", type=str, default="logistic",
                   choices=["logistic", "linear", "mlp_clf", "mlp_reg"])
    p.add_argument("--gf_max_iters", type=int, default=10)
    p.add_argument("--gf_C", type=float, default=50.0)
    p.add_argument("--gf_fairness", type=str, default="SP",
                   choices=["FP", "FN", "FPR", "FNR", "SP"])
    p.add_argument('--decision_threshold', type=float, default=0.5)

    # Multicalibration
    p.add_argument("--mc_alpha", type=float, default=0.1,
                   help="Multicalibration alpha (캘리브레이션 허용오차)")
    p.add_argument("--mc_lambda", type=float, default=0.1,
                   help="Multicalibration lambda (학습 스텝/규제 강도)")
    p.add_argument("--mc_max_iter", type=int, default=30)
    p.add_argument("--mc_randomized", action="store_true", default=True)
    p.add_argument("--mc_use_oracle", action="store_true", default=False)

    # Sequential Fairness
    p.add_argument("--seq_alpha", type=float, default=0.1,
                   help="Sequential Fairness의 alpha/step (기존 0.1을 인자로 노출)")
    p.add_argument("--seq_max_iter", type=int, default=50)

    # Results/logging
    p.add_argument("--exp_name", type=str, default=None)
    p.add_argument("--save_dir", type=str, default="results")
    # --- logging / UX ---
    p.add_argument("--log-interval", type=int, default=50,
                        help="train step마다 로그 찍는 간격")
    p.add_argument("--progress-bar", action="store_true",
                        help="tqdm 진행바 사용")
    p.add_argument("--logfile", type=str, default="",
                        help="로그를 파일에도 기록 (경로 지정)")
    p.add_argument("--heartbeat-secs", type=int, default=0,
                        help="N초마다 heartbeat 로그 (0이면 끔)")
    p.add_argument("--results-csv", type=str, default="results/all_runs.csv",
                   help="실험 결과를 누적 저장할 CSV 경로")

    return p.parse_args()


In [3]:
# Dataset & data options
dataset = ["toy_manyq", "adult", "communities", "dutch", "celebA"][1]  # "communities"
data_dir = ["../data/", "../data/raw/"][1]
q = [2, 3, 4, 5, 20, 50, 100][0]  # toy_manyq sensitive count

# X–S handling
x_sensitive = ["drop", "keep", "concat"][0]
sens_keys = ["race_basic", "sex,age,education_num", None][0]
sens_thresh = [0.25, 0.5, 0.75][1]

# Dataset-specific paths
dutch_path = ["data/raw/dutch.csv", "data/dutch_alt.csv"][0]
# communities_names = ["data/raw/communities.names", None][0]
# communities_data = ["data/raw/communities.data", None][0]
tfds_data_dir = [None, "data/tfds"][0]
celebA_manual_dir = [None, "data/raw/celebA"][0]

# Method selection
method = ["dr", "gerryfair", "multicalib", "sequential", "reduction"][0]

# Reduction baseline
red_constraint = ["DP", "EO"][0]
red_eps = [0.01, 0.02, 0.05][1]
red_max_iter = [10, 50, 100][1]
red_base = ["logreg", "linear_svm", "rf", "mlp_clf", "mlp_reg"][0]

# Common training hparams
seed = [0, 42, 123][1]
epochs = [100, 200, 300][2]
batch_size = [128, 256, 512][2]
lr = [1e-2, 1e-3, 1e-4][1]

# DR (ours)
lambda_fair = [0.0, 0.1, 1.0][0]
n_low = [50, 100, 200][1]
shrink_smallest_frac = [0.5, 1.0][1]
shrink_seed = [None, 42][0]
n_low_frac = [None, 0.01, 0.05][0]

# GerryFair
gamma = [0.001, 0.01, 0.1][1]
gf_base = ["logistic", "linear", "mlp_clf", "mlp_reg"][0]
gf_max_iters = [5, 10, 20][1]
gf_C = [10.0, 50.0, 100.0][1]
gf_fairness = ["FP", "FN", "FPR", "FNR", "SP"][4]
decision_threshold = [0.3, 0.5, 0.7][1]

# Multicalibration
mc_alpha = [0.05, 0.1, 0.2][1]
mc_lambda = [0.05, 0.1, 0.2][1]
mc_max_iter = [10, 30, 50][1]
mc_randomized = [True, False][0]
mc_use_oracle = [True, False][1]

# Sequential Fairness
seq_alpha = [0.05, 0.1, 0.2][1]
seq_max_iter = [10, 30, 50][1]

# Results/logging
exp_name = ["exp1", "exp2", None][2]
save_dir = ["../results", "outputs"][0]
log_interval = [10, 50, 100][1]
progress_bar = [True, False][0]
logfile = ["", "logs/run1.log"][0]
heartbeat_secs = [0, 30][0]
results_csv = ["../results/all_runs.csv", "../results/exp.csv"][0]


In [4]:
class Args:
    pass

args = Args()

# Dataset & data options
args.dataset = dataset
args.data_dir = data_dir
args.q = q

# X–S handling
args.x_sensitive = x_sensitive
args.sens_keys = sens_keys
args.sens_thresh = sens_thresh

# Dataset-specific paths
args.dutch_path = dutch_path
# args.communities_names = communities_names
# args.communities_data = communities_data
args.tfds_data_dir = tfds_data_dir
args.celebA_manual_dir = celebA_manual_dir

# Method selection
args.method = method

# Reduction baseline
args.red_constraint = red_constraint
args.red_eps = red_eps
args.red_max_iter = red_max_iter
args.red_base = red_base

# Common training hparams
args.seed = seed
args.epochs = epochs
args.batch_size = batch_size
args.lr = lr

# DR (ours)
args.lambda_fair = lambda_fair
args.n_low = n_low
args.shrink_smallest_frac = shrink_smallest_frac
args.shrink_seed = shrink_seed
args.n_low_frac = n_low_frac

# GerryFair
args.gamma = gamma
args.gf_base = gf_base
args.gf_max_iters = gf_max_iters
args.gf_C = gf_C
args.gf_fairness = gf_fairness
args.decision_threshold = decision_threshold

# Multicalibration
args.mc_alpha = mc_alpha
args.mc_lambda = mc_lambda
args.mc_max_iter = mc_max_iter
args.mc_randomized = mc_randomized
args.mc_use_oracle = mc_use_oracle

# Sequential Fairness
args.seq_alpha = seq_alpha
args.seq_max_iter = seq_max_iter

# Results/logging
args.exp_name = exp_name
args.save_dir = save_dir
args.log_interval = log_interval
args.progress_bar = progress_bar
args.logfile = logfile
args.heartbeat_secs = heartbeat_secs
args.results_csv = results_csv


In [5]:
import importlib
import fairbench.datasets

importlib.reload(fairbench)

import fairbench.datasets
from fairbench.utils.seed import set_seed
from fairbench.datasets import load_dataset   # 통합 로더
from fairbench.methods import run_method      # 통합 메서드 실행
from fairbench.metrics import compute_metrics 

In [ ]:
set_seed(seed)

# --- logger / heartbeat setup ---
from fairbench.utils.logging_utils import setup_logger, Timer, Heartbeat
log = setup_logger("fair", logfile or None)
hb = Heartbeat(log, heartbeat_secs)
hb.start()
log.info("args:\n" + json.dumps(vars(args), indent=2, ensure_ascii=False))


exp_name = exp_name or f"{dataset}_{method}_seed{seed}_{int(time.time())}"
save_dir = Path(save_dir); save_dir.mkdir(exist_ok=True, parents=True)
out_csv = save_dir / "all_results.csv"

# ---- 타이밍 측정 시작 ----
job_start_iso = datetime.now().isoformat(timespec="seconds")
t0_total = time.perf_counter()
prep_sec = np.nan
run_sec = np.nan
metrics_sec = np.nan

# 1) 데이터 로딩 + 2) 방법 실행
try:
    t0 = time.perf_counter()
    with Timer(log, "prepare_data"):
        data = load_dataset(args)
    prep_sec = time.perf_counter() - t0
    log.info("[data] prepared")

    t1 = time.perf_counter()
    with Timer(log, f"run_method:{method}"):
        pred_pack = run_method(args, data)
    run_sec = time.perf_counter() - t1
    log.info("[done] run_method finished")
finally:
    hb.stop()

# 3) 측정
t2 = time.perf_counter()
report = compute_metrics(args, data, pred_pack)
metrics_sec = time.perf_counter() - t2

# ---- 타이밍/메타 정보 구성 ----
total_sec = time.perf_counter() - t0_total
time_info = {
    "start_time": job_start_iso,
    "end_time": datetime.now().isoformat(timespec="seconds"),
    "time_prepare_data_sec": round(float(prep_sec), 4),
    "time_run_method_sec":   round(float(run_sec), 4),
    "time_metrics_sec":      round(float(metrics_sec), 4),
    "time_total_sec":        round(float(total_sec), 4),
}
if isinstance(pred_pack, dict) and ("note" in pred_pack):
    time_info["method_note"] = str(pred_pack["note"])

# data 로더가 meta를 담아줬다면 같이 기록(선택)
meta = (data.get("meta", {}) if isinstance(data, dict) else {}) or {}

def _as_str_list(x):
    if isinstance(x, (list, tuple)):
        return ",".join(map(str, x))
    return x



# 실험/페어니스 관련 하이퍼파라미터 모으기
hparams = {
    # 공통/식별
    "timestamp": datetime.now().isoformat(timespec="seconds"),
    "exp_name": exp_name,
    "dataset": dataset,
    "method": method,
    "seed": seed,

    # 데이터/민감속성 설정
    "x_sensitive": x_sensitive,
    "sens_keys": sens_keys,
    "sens_thresh": sens_thresh,

    # 학습 공통
    "epochs": epochs,
    "batch_size": batch_size,
    "lr": lr,

    # DR (ours)
    "lambda_fair": getattr(args, "lambda_fair", None),
    "n_low": getattr(args, "n_low", None),
    "lambda_fair": getattr(args, "lambda_fair", None),
    "n_low": getattr(args, "n_low", None),
    "n_low_frac": getattr(args, "n_low_frac", None),   # NEW
    "shrink_smallest_frac": getattr(args, "shrink_smallest_frac", None),  # NEW
    "shrink_seed": getattr(args, "shrink_seed", None),

    # GerryFair
    "gamma": getattr(args, "gamma", None),
    "gf_max_iters": getattr(args, "gf_max_iters", None),
    "gf_C": getattr(args, "gf_C", None),
    "gf_fairness": getattr(args, "gf_fairness", None),

    # Multicalibration
    "mc_alpha": getattr(args, "mc_alpha", None),
    "mc_lambda": getattr(args, "mc_lambda", None),
    "mc_max_iter": getattr(args, "mc_max_iter", None),
    "mc_randomized": getattr(args, "mc_randomized", None),
    "mc_use_oracle": getattr(args, "mc_use_oracle", None),

    # Sequential
    "seq_alpha": getattr(args, "seq_alpha", None),
    "seq_max_iter": getattr(args, "seq_max_iter", None),

    # Reduction (fairlearn ExponentiatedGradient)
    "red_constraint": getattr(args, "red_constraint", None),
    "red_eps": getattr(args, "red_eps", None),
    "red_max_iter": getattr(args, "red_max_iter", None),
    "red_base": getattr(args, "red_base", None),

    # 로더 메타(있으면 기록)
    "x_sensitive_mode": meta.get("x_sensitive_mode"),
    "used_S_cols": _as_str_list(meta.get("used_S_cols")),
    "dropped_cols": _as_str_list(meta.get("dropped_cols")),
}
# 타이밍 정보 합치기
hparams.update(time_info)

# 한 행(row)으로 합치기: report가 동일 키를 갖고 있으면 report 값을 우선
row = {**hparams, **report}

# 4) 저장
Path(out_csv).parent.mkdir(parents=True, exist_ok=True)
df = pd.DataFrame([row])
df.to_csv(out_csv, mode="a", header=not os.path.exists(out_csv), index=False)
print(f"[SAVE+APPEND] {out_csv} (+1 row)")
print(json.dumps(report, ensure_ascii=False, indent=2))

[15:01:24] args:
{
  "dataset": "adult",
  "data_dir": "../data/raw/",
  "q": 2,
  "x_sensitive": "drop",
  "sens_keys": "race_basic",
  "sens_thresh": 0.5,
  "dutch_path": "data/raw/dutch.csv",
  "tfds_data_dir": null,
  "celebA_manual_dir": null,
  "method": "dr",
  "red_constraint": "DP",
  "red_eps": 0.02,
  "red_max_iter": 50,
  "red_base": "logreg",
  "seed": 42,
  "epochs": 300,
  "batch_size": 512,
  "lr": 0.001,
  "lambda_fair": 0.0,
  "n_low": 100,
  "shrink_smallest_frac": 1.0,
  "shrink_seed": null,
  "n_low_frac": null,
  "gamma": 0.01,
  "gf_base": "logistic",
  "gf_max_iters": 10,
  "gf_C": 50.0,
  "gf_fairness": "SP",
  "decision_threshold": 0.5,
  "mc_alpha": 0.1,
  "mc_lambda": 0.1,
  "mc_max_iter": 30,
  "mc_randomized": true,
  "mc_use_oracle": false,
  "seq_alpha": 0.1,
  "seq_max_iter": 30,
  "exp_name": null,
  "save_dir": "../results",
  "log_interval": 50,
  "progress_bar": true,
  "logfile": "",
  "heartbeat_secs": 0,
  "results_csv": "../results/all_runs.csv"

[sens_keys] tokens=['race_basic']
[sens_keys] selected=['sex_Male', 'race_White', 'age_ge_40', 'married', 'edu_bachelor_plus', 'native_US', 'work_gov', 'work_private', 'occ_white_collar', 'occ_blue_collar', 'hours_ge_50', 'cap_gain_pos', 'cap_loss_pos', 'rel_husband']
[adult] S columns after filter = ['sex_Male', 'race_White', 'age_ge_40', 'married', 'edu_bachelor_plus', 'native_US', 'work_gov', 'work_private', 'occ_white_collar', 'occ_blue_collar', 'hours_ge_50', 'cap_gain_pos', 'cap_loss_pos', 'rel_husband']
[adult] final S_train cols = ['sex_Male', 'race_White', 'age_ge_40', 'married', 'edu_bachelor_plus', 'native_US', 'work_gov', 'work_private', 'occ_white_collar', 'occ_blue_collar', 'hours_ge_50', 'cap_gain_pos', 'cap_loss_pos', 'rel_husband']
[S|all] n=48842 cols=['sex_Male', 'race_White', 'age_ge_40', 'married', 'edu_bachelor_plus', 'native_US', 'work_gov', 'work_private', 'occ_white_collar', 'occ_blue_collar', 'hours_ge_50', 'cap_gain_pos', 'cap_loss_pos', 'rel_husband'] (aggre

[15:01:26] ✔ prepare_data done in 2.23s
[15:01:26] [data] prepared
[15:01:26] ▶ run_method:dr ...


[S] Saved subgroup INTERSECTION stats to CSV: datasets/sensitive_stats_adult.csv
  split                                           subgroup  count  proportion  \
0   all  not sex Male & not race White & not age ge 40 ...     21    0.000430   
1   all  not sex Male & not race White & not age ge 40 ...      0    0.000000   
2   all  not sex Male & not race White & not age ge 40 ...      3    0.000061   
3   all  not sex Male & not race White & not age ge 40 ...      0    0.000000   
4   all  not sex Male & not race White & not age ge 40 ...      0    0.000000   

       n                                            columns  
0  48842  sex_Male;race_White;age_ge_40;married;edu_bach...  
1  48842  sex_Male;race_White;age_ge_40;married;edu_bach...  
2  48842  sex_Male;race_White;age_ge_40;married;edu_bach...  
3  48842  sex_Male;race_White;age_ge_40;married;edu_bach...  
4  48842  sex_Male;race_White;age_ge_40;married;edu_bach...  


  0%|          | 0/300 [00:00<?, ?it/s]

[15:01:29] ✔ run_method:dr done in 2.46s
[15:01:29] [done] run_method finished


In [8]:
# def compute_metrics(args, data, pred_pack):
proba = pred_pack.get("proba", None)
pred  = pred_pack.get("pred",  None)

report = dict(dataset=getattr(args, "dataset", ""),
                method=getattr(args, "method", ""),
                seed=getattr(args, "seed", None))

thr = getattr(args, "thr", 0.5)
min_support = getattr(args, "min_support", 5)
mc_bins = getattr(args, "mc_bins", 10)
mc_min_support = getattr(args, "mc_min_support", 10)

# --- 정답 y 수집 ---
if data["type"] == "tabular":
    y = data.get("y_test", None)
    S = data.get("S_test", None)
else:
    ys, Ss = [], []
    for _, yb, Sb in data["test_loader"]:
        ys.append(yb.numpy())
        Ss += Sb
    y = np.concatenate(ys) if len(ys) > 0 else None
    S = Ss  # 이미지: list[dict]

In [10]:
S

,sex_bin
47991,0
46304,0
10268,0
30513,0
7330,0
...,...
9077,0
55575,0
12318,0
43502,0


ModuleNotFoundError: No module named 'HKRR'

In [ ]:
getattr(args, "seq_alpha", None)

0.1

In [7]:
from sklearn.model_selection import train_test_split
from fairbench.utils.trainval import split_train_val
import numpy as np

n = 20000; d = 20; q = 2
assert d > q, f"d must be larger than q, but got d={d}, q={q}"
rng = np.random.default_rng(args.seed)
X = rng.normal(size=(n, d)).astype(np.float32)

# latent sensitive factors (q binary), correlated with some features
S = (rng.normal(size=(n, q)) + 0.5*X[:, :min(d, q)][:, None]).mean(axis=1, keepdims=True)
S = (rng.normal(size=(n, q)) > 0.0).astype(int)  # 독립적인 q도 가능
S = pd.DataFrame(S, columns=[f"s{j}" for j in range(q)])

# target: non-linear function + mild spurious corr with a few sensitive attrs
logits = 0.8*X[:,0] - 0.6*X[:,1] + 0.5*X[:,2]*X[:,3] + 0.2*(S.values[:,:min(q,5)].sum(axis=1))
p = 1/(1+np.exp(-logits))
y = (rng.uniform(size=n) < p).astype(int)

X = pd.DataFrame(X, columns=[f"x{j}" for j in range(d)])

X_tr, X_te, y_tr, y_te, S_tr, S_te = train_test_split(X, y, S, test_size=0.2, random_state=args.seed, stratify=y)
X_tr, X_va, y_tr, y_va, S_tr, S_va = split_train_val(X_tr, y_tr, S_tr, val_size=0.2, seed=args.seed)


In [8]:
S

,s0,s1
0,0,1
1,0,1
2,0,0
3,1,0
4,1,1
...,...,...
19995,1,1
19996,0,1
19997,0,1
19998,1,0


In [11]:

n = 20000; d = 20; q = 2
assert d > q, f"d must be larger than q, but got d={d}, q={q}"
rng = np.random.default_rng(args.seed)
X = rng.normal(size=(n, d)).astype(np.float32)

(rng.normal(size=(n, q)) + 0.5*X[:, :min(d, q)][:, None]).mean(axis=1, keepdims=True)

array([[[ 0.13617472, -0.5246417 ]],

       [[-0.108615  , -0.34511442]],

       [[ 0.35544327,  0.26692747]],

       ...,

       [[ 0.04899138,  0.0732359 ]],

       [[-0.95700005, -0.67619325]],

       [[ 0.33115828,  0.19019868]]])

In [2]:
import pandas as pd

df = pd.read_csv("results_0909/dr_subgroup_subset_3q_adult/all_results.csv")

In [4]:
df.columns

Index(['timestamp', 'exp_name', 'dataset', 'method', 'seed', 'x_sensitive',
       'sens_keys', 'sens_thresh', 'epochs', 'batch_size', 'lr', 'lambda_fair',
       'n_low', 'n_low_frac', 'shrink_smallest_frac', 'shrink_seed', 'gamma',
       'gf_max_iters', 'gf_C', 'gf_fairness', 'mc_alpha', 'mc_lambda',
       'mc_max_iter', 'mc_randomized', 'mc_use_oracle', 'seq_alpha',
       'seq_max_iter', 'red_constraint', 'red_eps', 'red_max_iter', 'red_base',
       'x_sensitive_mode', 'used_S_cols', 'dropped_cols', 'start_time',
       'end_time', 'time_prepare_data_sec', 'time_run_method_sec',
       'time_metrics_sec', 'time_total_sec', 'V_stats', 'accuracy',
       'supipm_rbf', 'supipm_w1', 'sup_mmd_dfcols', 'sup_w1_dfcols',
       'sup_mmd_over_V', 'sup_w1_over_V', 'spd_worst', 'spd_mean',
       'worst_weighted_group_spd', 'mean_weighted_group_spd',
       'worst_spd_over_V', 'mean_spd_over_V', 'worst_weighted_spd_over_V',
       'mean_weighted_spd_over_V', 'fpr_worst', 'fpr_mean', 'mc_wo